# Sentiment Analysis Part 2

_Natural Langauge Processing Nanodegree Program_

---



## Step 5: Switching gears - RNNs

We just saw how the task of sentiment analysis can be solved via a traditional machine learning approach: BoW + a nonlinear classifier. We now switch gears and use Recurrent Neural Networks, and in particular LSTMs, to perform sentiment analysis in Keras. Conveniently, Keras has a built-in [IMDb movie reviews dataset](https://keras.io/datasets/#imdb-movie-reviews-sentiment-classification) that we can use, with the same vocabulary size.

In [1]:
from keras.datasets import imdb  # import the built-in imdb dataset in Keras

# Set the vocabulary size
vocabulary_size = 5000

# Load in training and test data (note the difference in convention compared to scikit-learn)
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocabulary_size)
print("Loaded dataset with {} training samples, {} test samples".format(len(X_train), len(X_test)))

Using TensorFlow backend.


17465344/17464789 [==============================] - 2s 0us/step
Loaded dataset with 25000 training samples, 25000 test samples


In [2]:
# Inspect a sample review and its label
print("--- Review ---")
print(X_train[7])
print("--- Label ---")
print(y_train[7])

--- Review ---
[1, 43, 3988, 44, 4, 3324, 1937, 155, 73, 25, 238, 1585, 3324, 25, 115, 124, 553, 51, 70, 13, 135, 13, 119, 14, 22, 12, 47, 15, 172, 281, 7, 1940, 5, 3093, 258, 11, 108, 40, 759, 34, 72, 5, 4, 2, 2, 4253, 2, 4, 936, 7, 2, 606, 183, 5, 89, 400, 75, 4909, 98, 4, 22, 1791, 178, 51, 45, 40, 8, 67, 4, 182, 143, 2843, 523, 5, 32, 4, 1236, 736, 5, 189, 36, 2, 2, 4253, 2423, 6, 787, 2, 406, 11, 94, 2, 15, 259, 871, 69, 6, 1548, 70, 199, 2198, 8, 5, 809, 11, 119, 19, 106, 12, 19, 369, 106, 12, 19, 447, 663, 1703, 6, 2, 2, 2]
--- Label ---
1


Notice that the label is an integer (0 for negative, 1 for positive), and the review itself is stored as a sequence of integers. These are word IDs that have been preassigned to individual words. To map them back to the original words, you can use the dictionary returned by `imdb.get_word_index()`.

In [3]:
# Map word IDs back to words
word2id = imdb.get_word_index()
id2word = {i: word for word, i in word2id.items()}
print("--- Review (with words) ---")
print([id2word.get(i, " ") for i in X_train[7]])
print("--- Label ---")
print(y_train[7])

1646592/1641221 [==============================] - 1s 0us/step
--- Review (with words) ---
['the', 'out', 'offering', 'has', 'of', 'arnold', 'youth', '10', 'much', 'have', '2', 'folks', 'arnold', 'have', 'best', 'does', 'killed', 'when', 'well', 'was', 'why', 'was', 'did', 'as', 'you', 'that', 'there', 'for', 'every', 'actor', 'br', 'partner', 'to', 'bugs', 'although', 'this', 'many', 'just', 'bunch', 'who', 'we', 'to', 'of', 'and', 'and', 'melting', 'and', 'of', 'fantasy', 'br', 'and', 'hell', 'seems', 'to', "don't", 'name', 'bad', 'recording', 'any', 'of', 'you', 'pacing', 'want', 'when', 'if', 'just', 'in', 'can', 'of', 'young', "i'm", 'experiment', 'directed', 'to', 'an', 'of', 'alive', 'appears', 'to', 'fact', 'from', 'and', 'and', 'melting', 'provided', 'is', '9', 'and', 'others', 'this', 'make', 'and', 'for', 'especially', "you've", 'me', 'is', 'forgotten', 'well', 'give', 'practically', 'in', 'to', 'form', 'this', 'did', 'film', 'character', 'that', 'film', 'women', 'character'

In [4]:
X_len = [len(l) for l in X_train]
print("Min review length: {} and Max length: {}".format(min(X_len), max(X_len)))

Min review length: 11 and Max length: 2494


Unlike our Bag-of-Words approach, where we simply summarized the counts of each word in a document, this representation essentially retains the entire sequence of words (minus punctuation, stopwords, etc.). This is critical for RNNs to function. But it also means that now the features can be of different lengths!

#### Question: Variable length reviews

What is the maximum review length (in terms of number of words) in the training set? What is the minimum?

#### Answer:

Min review length: 11 and Max length: 2494


### TODO: Pad sequences

In order to feed this data into your RNN, all input documents must have the same length. Let's limit the maximum review length to `max_words` by truncating longer reviews and padding shorter reviews with a null value (0). You can accomplish this easily using the [`pad_sequences()`](https://keras.io/preprocessing/sequence/#pad_sequences) function in Keras. For now, set `max_words` to 500.

In [5]:
#from keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences

# Set the maximum number of words per document (for both training and testing)
max_words = 500

# TODO: Pad sequences in X_train and X_test
X_train = pad_sequences(X_train, maxlen=max_words)
X_test = pad_sequences(X_test, maxlen=max_words)

In [12]:
import pprint
pp = pprint.PrettyPrinter()
pp.pprint(X_train)

array([[   0,    0,    0, ...,   10,  342,  158],
       [   0,    0,    0, ...,   23,    4, 1108],
       [   0,    0,    0, ...,  193,    6, 4548],
       ..., 
       [   0,    0,    0, ...,  737,    8,   32],
       [   0,    0,    0, ...,  479,    7,    2],
       [   0,    0,    0, ...,  956,    6,  991]], dtype=int32)


### TODO: Design an RNN model for sentiment analysis

Build your model architecture in the code cell below. We have imported some layers from Keras that you might need but feel free to use any other layers / transformations you like.

Remember that your input is a sequence of words (technically, integer word IDs) of maximum length = `max_words`, and your output is a binary sentiment label (0 or 1).

In [23]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

# TODO: Design your model
embedding_size = 32
model = Sequential()
model.add(Embedding(vocabulary_size, embedding_size, input_length=max_words))
model.add(LSTM(64))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
lstm_8 (LSTM)                (None, 64)                24832     
_________________________________________________________________
dropout_7 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 65        
Total params: 184,897
Trainable params: 184,897
Non-trainable params: 0
_________________________________________________________________
None


#### Question: Architecture and parameters

Briefly describe your neural net architecture. How many model parameters does it have that need to be trained?

#### Answer:

I have the following layers:
- an Embedding-Layer to vectorize our input-sequence
- a LSTM with the given output size
- a Dropout-Layer for regularization
- and a dense layer with a sigmoid activation to output between 0 and 1

Total params: 184,897

### TODO: Train and evaluate your model

Now you are ready to train your model. In Keras world, you first need to _compile_ your model by specifying the loss function and optimizer you want to use while training, as well as any evaluation metrics you'd like to measure. Specify the approprate parameters, including at least one metric `'accuracy'`.

In [25]:
# TODO: Compile your model, specifying a loss function, optimizer, and metrics
model.compile(optimizer="rmsprop",loss="binary_crossentropy", metrics=["accuracy"])

Once compiled, you can kick off the training process. There are two important training parameters that you have to specify - **batch size** and **number of training epochs**, which together with your model architecture determine the total training time.

Training may take a while, so grab a cup of coffee, or better, go for a hike! If possible, consider using a GPU, as a single training run can take several hours on a CPU.

> **Tip**: You can split off a small portion of the training set to be used for validation during training. This will help monitor the training process and identify potential overfitting. You can supply a validation set to `model.fit()` using its `validation_data` parameter, or just specify `validation_split` - a fraction of the training data for Keras to set aside for this purpose (typically 5-10%). Validation metrics are evaluated once at the end of each epoch.

In [31]:
# TODO: Specify training parameters: batch size and number of epochs
batch_size = 128
num_epochs = 6

# TODO(optional): Reserve/specify some training data for validation (not to be used for training)

# TODO: Train your model
model.fit(X_train, y_train, batch_size=batch_size, 
          epochs=num_epochs, verbose=1, validation_split=0.1)

Train on 22500 samples, validate on 2500 samples
Epoch 1/6
22500/22500 [==============================] - 278s 12ms/step - loss: 0.3138 - acc: 0.8744 - val_loss: 0.4129 - val_acc: 0.8580
Epoch 2/6
22500/22500 [==============================] - 277s 12ms/step - loss: 0.2717 - acc: 0.8932 - val_loss: 0.3164 - val_acc: 0.8720
Epoch 3/6
22500/22500 [==============================] - 278s 12ms/step - loss: 0.2480 - acc: 0.9037 - val_loss: 0.3606 - val_acc: 0.8456
Epoch 4/6
22500/22500 [==============================] - 278s 12ms/step - loss: 0.2323 - acc: 0.9116 - val_loss: 0.3074 - val_acc: 0.8820
Epoch 5/6
22500/22500 [==============================] - 278s 12ms/step - loss: 0.2150 - acc: 0.9184 - val_loss: 0.3097 - val_acc: 0.8672
Epoch 6/6
22500/22500 [==============================] - 278s 12ms/step - loss: 0.2030 - acc: 0.9243 - val_loss: 0.3283 - val_acc: 0.8688


In [36]:
import os
cache_dir = os.path.join("cache", "sentiment_analysis")
# Save your model, so that you can quickly load it in future (and perhaps resume training)
model_file = "rnn_model.h5"  # HDF5 file
model.save(os.path.join(cache_dir, "rnn_model.h5"))

OSError: Unable to create file (unable to open file: name = 'cache/sentiment_analysis/rnn_model.h5', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 242)

In [ ]:
# Later you can load it using keras.models.load_model()
#from keras.models import load_model
#model = load_model(os.path.join(cache_dir, model_file))

Once you have trained your model, it's time to see how well it performs on unseen test data.

In [35]:
# Evaluate your model on the test set
scores = model.evaluate(X_test, y_test, verbose=0)  # returns loss and other metrics specified in model.compile()
print("Test accuracy:", scores[1])  # scores[1] should correspond to accuracy if you passed in metrics=['accuracy']

Test accuracy: 0.8616


#### Question: Comparing RNNs and Traditional Methods

How well does your RNN model perform compared to the BoW + Gradient-Boosted Decision Trees?

#### Answer:

RNN Classifier test accuracy: 0.8616

GradientBoostingClassifier test accuracy: 0.8268

GaussianNB test accuracy: 0.72768

## Extensions

There are several ways in which you can build upon this notebook. Each comes with its set of challenges, but can be a rewarding experience.

- The first thing is to try and improve the accuracy of your model by experimenting with different architectures, layers and parameters. How good can you get without taking prohibitively long to train? How do you prevent overfitting?

- Then, you may want to deploy your model as a mobile app or web service. What do you need to do in order to package your model for such deployment? How would you accept a new review, convert it into a form suitable for your model, and perform the actual prediction? (Note that the same environment you used during training may not be available.)

- One simplification we made in this notebook is to limit the task to binary classification. The dataset actually includes a more fine-grained review rating that is indicated in each review's filename (which is of the form `<[id]_[rating].txt>` where `[id]` is a unique identifier and `[rating]` is on a scale of 1-10; note that neutral reviews > 4 or < 7 have been excluded). How would you modify the notebook to perform regression on the review ratings? In what situations is regression more useful than classification, and vice-versa?

Whatever direction you take, make sure to share your results and learnings with your peers, through blogs, discussions and participating in online competitions. This is also a great way to become more visible to potential employers!